In [8]:
#If you have trouble using Gurobi
#using Pkg
#Pkg.rm("Gurobi")
#Pkg.add("Gurobi")

    Updating `C:\Users\Admin\.julia\environments\v1.10\Project.toml`
  [2e9cd046] - Gurobi v1.2.3
    Updating `C:\Users\Admin\.julia\environments\v1.10\Manifest.toml`
  [2e9cd046] - Gurobi v1.2.3
  [4af54fe1] - LazyArtifacts
        Info We haven't cleaned this depot up for a bit, running Pkg.gc()...
      Active manifest files: 1 found
      Active artifact files: 5 found
      Active scratchspaces: 4 found
     Deleted no artifacts, repos, packages or scratchspaces
   Resolving package versions...
   Installed Gurobi ───── v1.3.0
   Installed Gurobi_jll ─ v11.0.2+2
    Updating `C:\Users\Admin\.julia\environments\v1.10\Project.toml`
  [2e9cd046] + Gurobi v1.3.0
    Updating `C:\Users\Admin\.julia\environments\v1.10\Manifest.toml`
  [2e9cd046] + Gurobi v1.3.0
  [c018c7e6] + Gurobi_jll v11.0.2+2
    Building Gurobi → `C:\Users\Admin\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\86bb48959ff82fdcf981c55462bd8427773ea6fc\build.log`
Precompiling project...
  ✓ Gurobi_jll
  ✓ G

In [14]:
using Gurobi, JuMP, Distributions

In [26]:
using JuMP
using Gurobi
using Distributions

# Create the model with Gurobi optimizer
m = Model(Gurobi.Optimizer)
n = 12
nt = 20

# Define the variables
@variable(m, W[0:n] >= 0)
@variable(m, H[1:n] >= 0, Int)
@variable(m, F[1:n] >= 0, Int)
@variable(m, I[0:n] >= 0)
@variable(m, P[1:n] >= 0)
@variable(m, O[1:n] >= 0)
@variable(m, U[1:n] >= 0)
@variable(m, S[1:n] >= 0)

# Define the distributions
d = Uniform(0, 100000)
d1 = Uniform(0, 100)

# Generate random coefficients
ci = rand(d)
cr = rand(d)
co = rand(d)
cu = rand(d)
cs = rand(d)

# Initialize W[0] and I[0]
W_init = rand(d)
I_init = rand(d)

# Define a parameter K
K = rand(d1)

# Define constants
cf = 173490
ch = 2406.6
T = 12 * rand(2:5)
D = rand(d, n)  # Generate an array of random values for D

# Add initial constraints for W[0] and I[0]
@constraint(m, W[0] == W_init)
@constraint(m, I[0] == I_init)

# Define the constraints
for i in 1:n
    @constraint(m, W[i] == W[i-1] + H[i] - F[i])
    @constraint(m, P[i] == K * nt * W[i] + O[i] - U[i])
    @constraint(m, I[i] == I[i-1] + P[i] + S[i] - D[i])
end

# Define the objective function
@objective(m, Min, 
    ch * sum(H[i] for i in 1:n) +
    cf * sum(F[i] for i in 1:n) +
    ci * sum(I[i] for i in 1:n) +
    co * sum(O[i] for i in 1:n) +
    cu * sum(U[i] for i in 1:n) +
    cs * sum(S[i] for i in 1:n)
)

# Optimize the model
optimize!(m)

# Print the results
for i in 1:n
    println("W[$i] = ", value(W[i]))
    println("H[$i] = ", value(H[i]))
    println("F[$i] = ", value(F[i]))
    println("I[$i] = ", value(I[i]))
    println("P[$i] = ", value(P[i]))
    println("O[$i] = ", value(O[i]))
    println("U[$i] = ", value(U[i]))
    println("S[$i] = ", value(S[i]))
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 38 rows, 98 columns and 146 nonzeros
Model fingerprint: 0x51b2e0a9
Variable types: 74 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+03, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+03, 1e+05]
Presolve removed 2 rows and 26 columns
Presolve time: 0.00s
Presolved: 36 rows, 72 columns, 118 nonzeros
Variable types: 48 continuous, 24 integer (0 binary)
Found heuristic solution: objective 1.964504e+12

Root relaxation: objective 6.948918e+09, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |   